Real time update of statistics is not needed when
1) the system only manipulates afew stocks,
2) the system only manipulates a few indicators,
3) the system only aims at predicting a single time percision,
since the calculation of 23 indicators for entire week's data on a single stock only takes 0.1 second.

But as time goes on, this might become crucial.

Use help(ta.function_name) in bash python to learn more about each indicator's calculation.

In [61]:
import pandas as pd
import numpy as np
import pandas_ta as ta

pd.options.display.max_rows = 1000

In [13]:
df = pd.read_csv('data/csv/bar_set_huge_20200101_20230412_BABA_indicator.csv', index_col = ['symbol', 'timestamp'])

In [35]:
close = df['close']
close_df = pd.DataFrame({'close': df['close']})


print(type(close))
print(close.dtypes)
print(type(close_df))
print(close_df.dtypes)

<class 'pandas.core.series.Series'>
float64
<class 'pandas.core.frame.DataFrame'>
close    float64
dtype: object


In [49]:
# EMA
length=10
def EMA(close, length = 10):
    ema = close.ewm(span=length, adjust=False).mean()
    return ema

ema = EMA(close_df, length)
# adjust=False
# sma=True
# if sma:
#     sma_nth = close[0:length].sum() / length
#     close[:length - 1] = np.NaN
#     close.iloc[length - 1] = sma_nth
# EMA = close.ewm(span=length, adjust=adjust).mean()
print(ema-close_df)

                                     close
symbol timestamp                          
BABA   2020-07-27 13:57:00+00:00  0.000000
       2020-07-27 13:58:00+00:00  0.098182
       2020-07-27 13:59:00+00:00 -0.242851
       2020-07-27 14:00:00+00:00 -0.505515
       2020-07-27 14:01:00+00:00 -0.536330
...                                    ...
       2023-04-11 19:55:00+00:00  0.042303
       2023-04-11 19:56:00+00:00  0.059157
       2023-04-11 19:57:00+00:00  0.146583
       2023-04-11 19:58:00+00:00  0.074932
       2023-04-11 19:59:00+00:00  0.110399

[243399 rows x 1 columns]


In [51]:
# DEMA
def DEMA(close = None, ema = None, length = 10):
    if close is None and ema is None:
        print("Error: one of close or ema must be provided")
        return None

    if ema is None:
        ema1 = EMA(close, length)
    else:
        ema1 = ema
    ema2 = EMA(ema1, length)
    dema = 2 * ema1 - ema2
    return dema


dema = DEMA(ema = ema)
print(dema - close_df)

                                     close
symbol timestamp                          
BABA   2020-07-27 13:57:00+00:00  0.000000
       2020-07-27 13:58:00+00:00  0.080331
       2020-07-27 13:59:00+00:00 -0.213302
       2020-07-27 14:00:00+00:00 -0.389426
       2020-07-27 14:01:00+00:00 -0.343834
...                                    ...
       2023-04-11 19:55:00+00:00 -0.014854
       2023-04-11 19:56:00+00:00  0.001636
       2023-04-11 19:57:00+00:00  0.072869
       2023-04-11 19:58:00+00:00  0.000996
       2023-04-11 19:59:00+00:00  0.029834

[243399 rows x 1 columns]


In [52]:
# TEMA
def TEMA(ema1, ema2, length = 10):
    ema3 = EMA(ema2, length)
    tema = 3 * (ema1 - ema2) + ema3
    return tema

tema = TEMA(ema1 = ema, ema2 = dema)
print(tema - close_df)

                                     close
symbol timestamp                          
BABA   2020-07-27 13:57:00+00:00  0.000000
       2020-07-27 13:58:00+00:00  0.166341
       2020-07-27 13:59:00+00:00 -0.358331
       2020-07-27 14:00:00+00:00 -0.946538
       2020-07-27 14:01:00+00:00 -1.252227
...                                    ...
       2023-04-11 19:55:00+00:00  0.207299
       2023-04-11 19:56:00+00:00  0.226719
       2023-04-11 19:57:00+00:00  0.376882
       2023-04-11 19:58:00+00:00  0.304411
       2023-04-11 19:59:00+00:00  0.363796

[243399 rows x 1 columns]


In [56]:
def variance(close, length = 300):
    var = close.rolling(length).var()
    return var

In [57]:
def STDEV(close, length=30, ddof=None, talib=None, offset=None, **kwargs):
    stdev = variance(close, length).apply(np.sqrt)
    return stdev

std = STDEV(close_df, length=10)
print(std)

                                     close
symbol timestamp                          
BABA   2020-07-27 13:57:00+00:00       NaN
       2020-07-27 13:58:00+00:00       NaN
       2020-07-27 13:59:00+00:00       NaN
       2020-07-27 14:00:00+00:00       NaN
       2020-07-27 14:01:00+00:00       NaN
...                                    ...
       2023-04-11 19:55:00+00:00  0.042230
       2023-04-11 19:56:00+00:00  0.046455
       2023-04-11 19:57:00+00:00  0.069530
       2023-04-11 19:58:00+00:00  0.067874
       2023-04-11 19:59:00+00:00  0.071881

[243399 rows x 1 columns]


In [60]:
def BBANDS(close, length=5, std=2, mamode='sma'):
    if mamode == 'sma':
        mid_band = close.rolling(length).mean()
    elif mamode == 'ema':
        mid_band = EMA(close, length)
    std = STDEV(close, length)
    upper_band = mid_band + std * std
    lower_band = mid_band - std * std
    bandwidth = (upper_band - lower_band) / mid_band
    percent = (close - lower_band) / (upper_band - lower_band)
    return lower_band, mid_band, upper_band, bandwidth, percent

lower, mid, upper, bandwidth, percent = BBANDS(close_df)
print(lower)
print(mid)
print(upper)
print(bandwidth)
print(percent)

                                       close
symbol timestamp                            
BABA   2020-07-27 13:57:00+00:00         NaN
       2020-07-27 13:58:00+00:00         NaN
       2020-07-27 13:59:00+00:00         NaN
       2020-07-27 14:00:00+00:00         NaN
       2020-07-27 14:01:00+00:00  248.801670
...                                      ...
       2023-04-11 19:55:00+00:00   99.950532
       2023-04-11 19:56:00+00:00   99.937305
       2023-04-11 19:57:00+00:00   99.895600
       2023-04-11 19:58:00+00:00   99.873930
       2023-04-11 19:59:00+00:00   99.844132

[243399 rows x 1 columns]
                                    close
symbol timestamp                         
BABA   2020-07-27 13:57:00+00:00      NaN
       2020-07-27 13:58:00+00:00      NaN
       2020-07-27 13:59:00+00:00      NaN
       2020-07-27 14:00:00+00:00      NaN
       2020-07-27 14:01:00+00:00  248.961
...                                   ...
       2023-04-11 19:55:00+00:00   99.951
       202